#Setup

In [1]:
!pip install faker unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 1.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.6/265.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.1/138.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393

In [2]:
import os
import json
import pandas as pd
import numpy as np
from typing import Dict, List, Any, Tuple
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import classification_report, confusion_matrix
import random
from faker import Faker  # Used for generating mock data
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import time
# Import Unsloth and Peft for the fine-tuned model
from unsloth import FastLanguageModel
from peft import PeftModel

<ipython-input-2-974303e8781d>:16: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
class PersonalizedContext:
    """Class to store and process personalized context information."""
    def __init__(self, name: str, email: str, job_position: str, recent_activities: List[str]):
        self.name = name
        self.email = email
        self.job_position = job_position
        self.recent_activities = recent_activities

    def to_prompt_snippet(self) -> str:
        """Convert personal context to a snippet for the LLM prompt."""
        activities = "\n".join([f"- {activity}" for activity in self.recent_activities])
        return f"""
          Name: {self.name}
          Email: {self.email}
          Job Position: {self.job_position}

          Recent Activities (use these to make the email highly relevant):
          {activities}
          """

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'PersonalizedContext':
        """Create context from dictionary."""
        return cls(
            name=data.get('name', ''),
            email=data.get('email', ''),
            job_position=data.get('job_position', ''),
            recent_activities=data.get('recent_activities', [])
        )

In [4]:
prompt_head = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generate a convincing phishing email based on the given personal context.

### Input:
Using the following personal details, craft a convincing phishing email that leverages the person's job position and recent activities. The email should create urgency and prompt the user to click a link or perform a similar malicious action.

"""

prompt_tail = """\nGenerate ONLY the text of the phishing email, starting with the 'Subject:' line. Ensure it is well-formatted and appears like a real email.

### Response:"""

# Llama3.2-1B

In [6]:
# Define configuration
class PipelineConfig:
    """Configuration for the phishing generation and detection pipeline."""
    def __init__(self):
        # --- Fine-tuned LLM Settings ---
        self.base_model_id = "unsloth/llama-3.2-1b-Instruct"  # Updated to match your model
        self.lora_path = "kxm1k4m1/Llama3.2-1B-Instruct-Phishing"  # Your fine-tuned model
        self.max_seq_length = 4096

        # --- Phishing Detection Settings ---
        # Original model
        self.detection_model_name = "dima806/phishing-email-detection"
        # Additional models
        self.detection_model_name2 = "cybersectony/phishing-email-detection-distilbert_v2.4.1"
        self.detection_model_name3 = "ealvaradob/bert-finetuned-phishing"
        self.detection_model_device = "cuda" if torch.cuda.is_available() else "cpu"

        # --- Experiment Settings ---
        self.input_data_path = "personalized_contexts.csv"
        self.results_path = "llama3.2_1b.csv"
        self.sample_size = 20  # Number of emails to generate (adjust as needed)

        # --- LLM Generation Parameters ---
        self.max_output_tokens = 256
        self.temperature = 0.8
        self.top_p = 0.9
        self.seed = 42  # Seed for reproducibility in sampling

    def to_dict(self) -> Dict[str, Any]:
        """Convert config to dictionary for serialization"""
        return {k: v for k, v in self.__dict__.items() if not k.startswith('_')}

class FineTunedLLMPhishingGenerator:
    """Class to generate phishing emails using a fine-tuned LLM."""
    def __init__(self, config: PipelineConfig):
        self.config = config

        print(f"Loading fine-tuned LLM model: {config.base_model_id} with adapter: {config.lora_path}")
        try:
            # Load tokenizer
            self.tokenizer = AutoTokenizer.from_pretrained(config.base_model_id, use_fast=True)
            self.tokenizer.pad_token = self.tokenizer.eos_token  # Avoid padding issues

            # Load base model with optimizations
            self.model, _ = FastLanguageModel.from_pretrained(
                model_name=config.base_model_id,
                max_seq_length=config.max_seq_length,
                dtype=torch.float16,
                load_in_4bit=True,
            )

            # Load the LoRA adapter using PeftModel
            from peft import PeftModel
            self.model = PeftModel.from_pretrained(self.model, config.lora_path).to(self.model.device)

            # Rewrap for optimized inference
            self.model = FastLanguageModel.for_inference(self.model)

            print(f"Successfully loaded fine-tuned model and adapter")
            self.model_loaded = True
        except Exception as e:
            print(f"Error loading fine-tuned model: {e}")
            self.model_loaded = False

    def generate_phishing_email(self, context: PersonalizedContext) -> str:
        """Generate a phishing email using the provided context via fine-tuned LLM."""
        if not self.model_loaded:
            return "Error: Model failed to load. Cannot generate phishing email."

        try:
            prompt = self._create_phishing_prompt(context)
            print(f"Generating phishing email for {context.name} using fine-tuned LLM...")

            # Tokenize input
            input_ids = self.tokenizer(prompt, return_tensors="pt").input_ids.to(self.model.device)

            # Generate text
            output_ids = self.model.generate(
                input_ids,
                max_new_tokens=self.config.max_output_tokens,
                do_sample=True,
                temperature=self.config.temperature,
                top_p=self.config.top_p
            )

            # Decode the generated text
            full_response = self.tokenizer.decode(output_ids[0], skip_special_tokens=True)

            # Extract only the response part (after "### Response:")
            email_text = full_response.split("### Response:")[-1].strip()

            print(f"Successfully generated email via fine-tuned LLM")
            return email_text

        except Exception as e:
            error_message = f"Error during LLM generation for {context.name}: {e}"
            print(error_message)
            return f"Generation Error: {str(e)}"

    def _create_phishing_prompt(self, context: PersonalizedContext) -> str:
        """Create prompt for the fine-tuned LLM."""
        personal_context_snippet = context.to_prompt_snippet()

        prompt = prompt_head + personal_context_snippet + prompt_tail

        return prompt


class MultiModelPhishingDetector:
    """Class to detect phishing emails using multiple open-source models."""
    def __init__(self, config: PipelineConfig):
        self.config = config
        self.device = config.detection_model_device

        # Dictionary to store model information
        self.models = {
            "model1": {
                "name": config.detection_model_name,
                "tokenizer": None,
                "model": None,
                "loaded": False
            },
            "model2": {
                "name": config.detection_model_name2,
                "tokenizer": None,
                "model": None,
                "loaded": False
            },
            "model3": {
                "name": config.detection_model_name3,
                "tokenizer": None,
                "model": None,
                "loaded": False
            }
        }

        # Load all models
        for model_key, model_info in self.models.items():
            try:
                print(f"\nLoading detection {model_key}: {model_info['name']}...")
                model_info["tokenizer"] = AutoTokenizer.from_pretrained(model_info["name"])
                model_info["model"] = AutoModelForSequenceClassification.from_pretrained(model_info["name"])
                model_info["model"].to(self.device)
                model_info["model"].eval()  # Set model to evaluation mode
                model_info["loaded"] = True
                print(f"Successfully loaded {model_info['name']} to {self.device}")
            except Exception as e:
                print(f"Error loading detection model '{model_info['name']}': {e}")
                print(f"{model_key} will not be available for detection.")

    def detect_phishing(self, email_text: str) -> Dict[str, Any]:
        """Detect if an email is a phishing attempt using all available models."""
        results = {
            "model1_pred": False,
            "model1_score": 0.0,
            "model2_pred": False,
            "model2_score": 0.0,
            "model3_pred": False,
            "model3_score": 0.0
        }

        # Process with each model
        for model_key, model_info in self.models.items():
            if not model_info["loaded"]:
                print(f"{model_key} not loaded. Skipping detection with this model.")
                continue

            try:
                inputs = model_info["tokenizer"](
                    email_text,
                    return_tensors="pt",
                    truncation=True,
                    padding=True,
                    max_length=512  # Limit input length
                )
                inputs = {k: v.to(self.device) for k, v in inputs.items()}

                with torch.no_grad():
                    outputs = model_info["model"](**inputs)

                # Get prediction - handling both binary and multi-class cases
                probabilities = torch.softmax(outputs.logits, dim=1)

                # Check if this is a binary classification model
                if probabilities.shape[1] == 2:  # Binary: [not_phishing, phishing]
                    phishing_prob = probabilities[0, 1].item()  # Probability of the 'phishing' class
                else:  # Assume multi-class with phishing as one of the classes
                    # This is a simplified assumption - different models might have different label mappings
                    # You may need to adjust this based on the specific model's output format
                    phishing_prob = torch.max(probabilities).item()  # Use max probability as a simplification

                # Threshold for classification
                is_phishing = phishing_prob > 0.5  # Standard threshold

                # Store results based on model key
                results[f"{model_key}_pred"] = is_phishing
                results[f"{model_key}_score"] = phishing_prob

                print(f"{model_key} detection result: {'PHISHING' if is_phishing else 'NOT PHISHING'} (score: {phishing_prob:.4f})")

            except Exception as e:
                print(f"Error during phishing detection with {model_key}: {e}")
                # Results for this model will remain default (False, 0.0)

        return results


class ExperimentRunner:
    """Class to run the full phishing experiment pipeline and evaluate results."""
    def __init__(self, config: PipelineConfig):
        # Change to use the fine-tuned LLM generator
        self.generator = FineTunedLLMPhishingGenerator(config)
        self.detector = MultiModelPhishingDetector(config)  # Now using multi-model detector
        self.config = config # Keep config reference

    def load_contexts(self) -> List[PersonalizedContext]:
        """Load personalized contexts from CSV or generate mock data if file doesn't exist."""
        try:
            # Check if the file exists
            if not os.path.exists(self.config.input_data_path):
                print(f"Input file '{self.config.input_data_path}' not found. Generating mock data...")
                self._generate_mock_data(self.config.sample_size)

            data = pd.read_csv(self.config.input_data_path)
            contexts = []

            print(f"Loading {min(self.config.sample_size, len(data))} contexts from {self.config.input_data_path}")

            # Limit to sample size
            for _, row in data.head(self.config.sample_size).iterrows():
                # Parse activities from JSON string if stored that way
                activities = row['recent_activities']
                if isinstance(activities, str):
                    try:
                        # Assuming activities are stored as a JSON list string
                        activities = json.loads(activities)
                        # Ensure it's a list, handle cases where it might be a simple string
                        if not isinstance(activities, list):
                            activities = [str(activities)]  # Treat as a single activity if not a list
                    except json.JSONDecodeError:
                        # Handle cases where it's a simple string that isn't JSON
                        activities = [str(activities)]
                elif not isinstance(activities, list):
                    # Handle case where it's not a string or list (e.g., NaN)
                    activities = []

                context = PersonalizedContext(
                    name=row['name'],
                    email=row['email'],
                    job_position=row['job_position'],
                    recent_activities=activities
                )
                contexts.append(context)

            if not contexts:
                print("No contexts loaded. Generating sample contexts.")
                return self._generate_sample_contexts()

            return contexts

        except Exception as e:
            print(f"Error loading contexts from CSV: {e}")
            print("Generating sample contexts for demonstration...")
            return self._generate_sample_contexts()

    def _generate_mock_data(self, num_samples: int):
      """Generate mock data file with Faker."""
      fake = Faker()

      print("Generating synthetic personalized contexts...")

      # Number of records to generate (fixed to 100)

      # Set seeds for reproducibility
      random.seed(42)
      np.random.seed(42)
      Faker.seed(42)

      # Define common job positions
      job_positions = [
        "Software Engineer", "Product Manager", "Marketing Specialist",
        "HR Manager", "Financial Analyst", "Sales Representative",
        "Customer Support", "Data Scientist", "IT Administrator",
        "Project Manager", "Operations Manager", "Executive Assistant",
        "UX Designer", "DevOps Engineer", "Cybersecurity Analyst",
        "Business Analyst", "Legal Consultant", "Recruiter",
        "Quality Assurance Engineer", "Technical Writer", "AI Researcher",
        "Cloud Solutions Architect", "Network Engineer", "Growth Manager",
        "Mobile App Developer", "Systems Analyst", "Machine Learning Engineer",
        "Corporate Trainer", "Content Strategist", "Public Relations Officer",
        "Procurement Specialist", "Risk Manager", "Compliance Officer",
        "Information Security Officer", "Facilities Manager", "Product Designer",
        "Front-End Developer", "Back-End Developer", "Full Stack Developer",
        "Customer Success Manager"
      ]

      # Define common activity templates
      activity_templates = [
        "Working on the {} project",
        "Preparing for the {} presentation",
        "Reviewing {} documents",
        "Attending {} meeting",
        "Planning the next {} initiative",
        "Analyzing {} data trends",
        "Coordinating with the {} team",
        "Implementing a new {} system",
        "Researching {} solutions",
        "Drafting a {} proposal",
        "Responding to {} inquiries",
        "Conducting {} interviews",
        "Troubleshooting {} issues",
        "Organizing the {} workshop",
        "Setting up {} infrastructure",
        "Reviewing feedback from {} clients",
        "Deploying the latest {} update",
        "Refining the {} workflow",
        "Training new hires on {} tools",
        "Budgeting for the {} campaign",
        "Collaborating with {} partners",
        "Finalizing the {} contract",
        "Writing documentation for {} systems",
        "Prototyping the new {} feature",
        "Debugging {} module integration",
        "Evaluating {} vendor performance",
        "Optimizing {} pipeline efficiency"
      ]

      # Company domains
      domains = ["company.com", "enterprise.org", "techcorp.io", "globalfirm.co", "industryco.net"]

      # Generate data
      data = []
      for _ in range(num_samples):
          first_name = fake.first_name()
          last_name = fake.last_name()
          full_name = f"{first_name} {last_name}"

          domain = random.choice(domains)
          # Create plausible email
          email = f"{first_name.lower()}.{last_name.lower()}@{domain}"
          if random.random() < 0.2:  # Occasionally use a different format
              email = f"{first_name.lower()}{last_name.lower()[0]}@{domain}"

          job_position = random.choice(job_positions)

          # Generate 1-3 activities
          num_activities = random.randint(1, 3)
          activities = []
          for _ in range(num_activities):
              activity_template = random.choice(activity_templates)
              activity = activity_template.format(fake.bs())  # Use fake business phrases
              activities.append(activity)

          entry = {
              "name": full_name,
              "email": email,
              "job_position": job_position,
              "recent_activities": json.dumps(activities)
          }

          data.append(entry)

      # Create DataFrame and save to CSV
      df = pd.DataFrame(data)
      df.to_csv(self.config.input_data_path, index=False)

      print(f"Generated {num_samples} mock contexts and saved to {self.config.input_data_path}")

    def run_experiment(self) -> pd.DataFrame:
        """Run the full experiment pipeline: load, generate, detect, save."""
        # Step 1: Load or generate contexts
        print("\n--- Step 1: Loading personalized contexts ---")
        contexts = self.load_contexts()
        if not contexts:
            print("No contexts available to process. Exiting.")
            return pd.DataFrame()  # Return empty DataFrame

        print(f"Loaded {len(contexts)} contexts for processing")

        # Show a few examples
        print("\nExample contexts:")
        for i, context in enumerate(contexts[:min(len(contexts), 3)]):  # Show up to 3 examples
            print(f"\nContext {i+1}:")
            print(f"  Name: {context.name}")
            print(f"  Job: {context.job_position}")
            print(f"  Activities: {', '.join(context.recent_activities) if context.recent_activities else 'None'}")

        results = []

        # Step 2: Generate and detect emails
        print("\n--- Step 2: Generating and detecting phishing emails ---")
        processed_count = 0
        for i, context in enumerate(contexts):
            print(f"\nProcessing context {i+1}/{len(contexts)}: {context.name}")

            # Generate phishing email
            print(f"  Generating phishing email via fine-tuned LLM...")
            phishing_email = self.generator.generate_phishing_email(context)

            # --- Increment counter ---
            processed_count += 1
            print(processed_count)

            if not phishing_email or "Error:" in phishing_email:
                print(f"  Generation failed for {context.name}. Skipping detection.")
                result = {
                    "name": context.name,
                    "email": context.email,
                    "job_position": context.job_position,
                    "recent_activities": context.recent_activities,
                    "generated_email": phishing_email,  # Store error message
                    "model1_pred": False,  # Default values for failed generation
                    "model1_score": 0.0,
                    "model2_pred": False,
                    "model2_score": 0.0,
                    "model3_pred": False,
                    "model3_score": 0.0,
                    "true_label": True  # Still a phishing attempt conceptually
                }
                results.append(result)
                continue

            # Display truncated email preview
            preview = phishing_email.replace('\n', ' ').strip()
            preview = (preview[:150] + '...') if len(preview) > 150 else preview
            print(f"  Email preview: \"{preview}\"")

            # Detect if it's phishing with all models
            print(f"  Running phishing detection with multiple models...")
            detection_results = self.detector.detect_phishing(phishing_email)

            # Print a summary of detection results
            models_detected = sum(1 for k, v in detection_results.items() if k.endswith('_pred') and v)
            print(f"  Detection summary: {models_detected}/3 models identified as phishing")

            # Create result dictionary with all detection results
            result = {
                "name": context.name,
                "email": context.email,
                "job_position": context.job_position,
                "recent_activities": context.recent_activities,
                "generated_email": phishing_email,
                "true_label": True,  # We know it's phishing since we generated it
                **detection_results  # Unpack all detection results
            }
            results.append(result)

        if not results:
            print("No emails were generated successfully to analyze.")
            return pd.DataFrame()

        # Step 3: Create DataFrame and save results
        print("\n--- Step 3: Saving and analyzing results ---")
        results_df = pd.DataFrame(results)
        results_df.to_csv(self.config.results_path, index=False)
        print(f"Results saved to {self.config.results_path}")

        # Print detection summary for all models
        valid_emails = results_df[~results_df['generated_email'].str.contains("Error:", na=False)]
        if not valid_emails.empty:
            print("\n--- Detection Performance Summary ---")
            print(f"Total valid emails: {len(valid_emails)}")
            for model_num in range(1, 4):
                model_col = f"model{model_num}_pred"
                if model_col in valid_emails.columns:
                    detection_rate = valid_emails[model_col].mean() * 100
                    print(f"Model {model_num} detection rate: {detection_rate:.2f}%")

        return results_df

def main():
    """Main function to run the pipeline and evaluate results."""
    print("Starting Fine-Tuned LLM Phishing Email Generation and Detection Pipeline...")

    # Initialize configuration
    config = PipelineConfig()

    # Set a smaller sample size for testing
    config.sample_size = 100  # Generate fewer emails for testing

    # Create experiment runner
    runner = ExperimentRunner(config)

    # Run the experiment
    print("\nRunning experiment...")
    results_df = runner.run_experiment()

    if results_df.empty or results_df[~results_df['generated_email'].str.contains("Error:", na=False)].empty:
        print("\nExperiment finished but no valid emails were generated or processed for analysis.")
        return


if __name__ == "__main__":
    main()

Starting Fine-Tuned LLM Phishing Email Generation and Detection Pipeline...
Loading fine-tuned LLM model: unsloth/llama-3.2-1b-Instruct with adapter: kxm1k4m1/Llama3.2-1B-Instruct-Phishing
==((====))==  Unsloth 2025.5.6: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Successfully loaded fine-tuned model and adapter

Loading detection model1: dima806/phishing-email-detection...
Successfully loaded dima806/phishing-email-detection to cuda

Loading detection model2: cybersectony/phishing-email-detection-distilbert_v2.4.1...
Successfully loaded cybersectony/phishing-email-detection-distilbert_v2.4.1 to cuda

Loading detection

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Successfully loaded ealvaradob/bert-finetuned-phishing to cuda

Running experiment...

--- Step 1: Loading personalized contexts ---
Input file 'personalized_contexts.csv' not found. Generating mock data...
Generating synthetic personalized contexts...
Generated 10 mock contexts and saved to personalized_contexts.csv
Loading 10 contexts from personalized_contexts.csv
Loaded 10 contexts for processing

Example contexts:

Context 1:
  Name: Danielle Johnson
  Job: Recruiter
  Activities: Implementing a new empower interactive e-services system

Context 2:
  Name: Donald Garcia
  Job: Facilities Manager
  Activities: Training new hires on extend e-business applications tools

Context 3:
  Name: Robert Johnson
  Job: Sales Representative
  Activities: Implementing a new architect bleeding-edge mindshare system

--- Step 2: Generating and detecting phishing emails ---

Processing context 1/10: Danielle Johnson
  Generating phishing email via fine-tuned LLM...
Generating phishing email for D

In [7]:
# Define configuration
class PipelineConfig:
    """Configuration for the phishing generation and detection pipeline."""
    def __init__(self):
        # --- Fine-tuned LLM Settings ---
        self.base_model_id = "unsloth/Qwen3-0.6B-unsloth-bnb-4bit"  # Updated to match your model
        self.lora_path = "kxm1k4m1/Qwen3-0.6B-unsloth-bnb-4bit-Phishing"  # Your fine-tuned model
        self.max_seq_length = 4096

        # --- Phishing Detection Settings ---
        # Original model
        self.detection_model_name = "dima806/phishing-email-detection"
        # Additional models
        self.detection_model_name2 = "cybersectony/phishing-email-detection-distilbert_v2.4.1"
        self.detection_model_name3 = "ealvaradob/bert-finetuned-phishing"
        self.detection_model_device = "cuda" if torch.cuda.is_available() else "cpu"

        # --- Experiment Settings ---
        self.input_data_path = "personalized_contexts.csv"
        self.results_path = "qwen3_0.6B.csv"
        self.sample_size = 20  # Number of emails to generate (adjust as needed)

        # --- LLM Generation Parameters ---
        self.max_output_tokens = 256
        self.temperature = 0.8
        self.top_p = 0.9
        self.seed = 42  # Seed for reproducibility in sampling

    def to_dict(self) -> Dict[str, Any]:
        """Convert config to dictionary for serialization"""
        return {k: v for k, v in self.__dict__.items() if not k.startswith('_')}

class FineTunedLLMPhishingGenerator:
    """Class to generate phishing emails using a fine-tuned LLM."""
    def __init__(self, config: PipelineConfig):
        self.config = config

        print(f"Loading fine-tuned LLM model: {config.base_model_id} with adapter: {config.lora_path}")
        try:
            # Load tokenizer
            self.tokenizer = AutoTokenizer.from_pretrained(config.base_model_id, use_fast=True)
            self.tokenizer.pad_token = self.tokenizer.eos_token  # Avoid padding issues

            # Load base model with optimizations
            self.model, _ = FastLanguageModel.from_pretrained(
                model_name=config.base_model_id,
                max_seq_length=config.max_seq_length,
                dtype=torch.float16,
                load_in_4bit=True,
            )

            # Load the LoRA adapter using PeftModel
            from peft import PeftModel
            self.model = PeftModel.from_pretrained(self.model, config.lora_path).to(self.model.device)

            # Rewrap for optimized inference
            self.model = FastLanguageModel.for_inference(self.model)

            print(f"Successfully loaded fine-tuned model and adapter")
            self.model_loaded = True
        except Exception as e:
            print(f"Error loading fine-tuned model: {e}")
            self.model_loaded = False

    def generate_phishing_email(self, context: PersonalizedContext) -> str:
        """Generate a phishing email using the provided context via fine-tuned LLM."""
        if not self.model_loaded:
            return "Error: Model failed to load. Cannot generate phishing email."

        try:
            prompt = self._create_phishing_prompt(context)
            print(f"Generating phishing email for {context.name} using fine-tuned LLM...")

            # Tokenize input
            input_ids = self.tokenizer(prompt, return_tensors="pt").input_ids.to(self.model.device)

            # Generate text
            output_ids = self.model.generate(
                input_ids,
                max_new_tokens=self.config.max_output_tokens,
                do_sample=True,
                temperature=self.config.temperature,
                top_p=self.config.top_p
            )

            # Decode the generated text
            full_response = self.tokenizer.decode(output_ids[0], skip_special_tokens=True)

            # Extract only the response part (after "### Response:")
            email_text = full_response.split("### Response:")[-1].strip()

            print(f"Successfully generated email via fine-tuned LLM")
            return email_text

        except Exception as e:
            error_message = f"Error during LLM generation for {context.name}: {e}"
            print(error_message)
            return f"Generation Error: {str(e)}"

    def _create_phishing_prompt(self, context: PersonalizedContext) -> str:
        """Create prompt for the fine-tuned LLM."""
        personal_context_snippet = context.to_prompt_snippet()

        prompt = prompt_head + personal_context_snippet + prompt_tail

        return prompt


class MultiModelPhishingDetector:
    """Class to detect phishing emails using multiple open-source models."""
    def __init__(self, config: PipelineConfig):
        self.config = config
        self.device = config.detection_model_device

        # Dictionary to store model information
        self.models = {
            "model1": {
                "name": config.detection_model_name,
                "tokenizer": None,
                "model": None,
                "loaded": False
            },
            "model2": {
                "name": config.detection_model_name2,
                "tokenizer": None,
                "model": None,
                "loaded": False
            },
            "model3": {
                "name": config.detection_model_name3,
                "tokenizer": None,
                "model": None,
                "loaded": False
            }
        }

        # Load all models
        for model_key, model_info in self.models.items():
            try:
                print(f"\nLoading detection {model_key}: {model_info['name']}...")
                model_info["tokenizer"] = AutoTokenizer.from_pretrained(model_info["name"])
                model_info["model"] = AutoModelForSequenceClassification.from_pretrained(model_info["name"])
                model_info["model"].to(self.device)
                model_info["model"].eval()  # Set model to evaluation mode
                model_info["loaded"] = True
                print(f"Successfully loaded {model_info['name']} to {self.device}")
            except Exception as e:
                print(f"Error loading detection model '{model_info['name']}': {e}")
                print(f"{model_key} will not be available for detection.")

    def detect_phishing(self, email_text: str) -> Dict[str, Any]:
        """Detect if an email is a phishing attempt using all available models."""
        results = {
            "model1_pred": False,
            "model1_score": 0.0,
            "model2_pred": False,
            "model2_score": 0.0,
            "model3_pred": False,
            "model3_score": 0.0
        }

        # Process with each model
        for model_key, model_info in self.models.items():
            if not model_info["loaded"]:
                print(f"{model_key} not loaded. Skipping detection with this model.")
                continue

            try:
                inputs = model_info["tokenizer"](
                    email_text,
                    return_tensors="pt",
                    truncation=True,
                    padding=True,
                    max_length=512  # Limit input length
                )
                inputs = {k: v.to(self.device) for k, v in inputs.items()}

                with torch.no_grad():
                    outputs = model_info["model"](**inputs)

                # Get prediction - handling both binary and multi-class cases
                probabilities = torch.softmax(outputs.logits, dim=1)

                # Check if this is a binary classification model
                if probabilities.shape[1] == 2:  # Binary: [not_phishing, phishing]
                    phishing_prob = probabilities[0, 1].item()  # Probability of the 'phishing' class
                else:  # Assume multi-class with phishing as one of the classes
                    # This is a simplified assumption - different models might have different label mappings
                    # You may need to adjust this based on the specific model's output format
                    phishing_prob = torch.max(probabilities).item()  # Use max probability as a simplification

                # Threshold for classification
                is_phishing = phishing_prob > 0.5  # Standard threshold

                # Store results based on model key
                results[f"{model_key}_pred"] = is_phishing
                results[f"{model_key}_score"] = phishing_prob

                print(f"{model_key} detection result: {'PHISHING' if is_phishing else 'NOT PHISHING'} (score: {phishing_prob:.4f})")

            except Exception as e:
                print(f"Error during phishing detection with {model_key}: {e}")
                # Results for this model will remain default (False, 0.0)

        return results


class ExperimentRunner:
    """Class to run the full phishing experiment pipeline and evaluate results."""
    def __init__(self, config: PipelineConfig):
        # Change to use the fine-tuned LLM generator
        self.generator = FineTunedLLMPhishingGenerator(config)
        self.detector = MultiModelPhishingDetector(config)  # Now using multi-model detector
        self.config = config # Keep config reference

    def load_contexts(self) -> List[PersonalizedContext]:
        """Load personalized contexts from CSV or generate mock data if file doesn't exist."""
        try:
            # Check if the file exists
            if not os.path.exists(self.config.input_data_path):
                print(f"Input file '{self.config.input_data_path}' not found. Generating mock data...")
                self._generate_mock_data(self.config.sample_size)

            data = pd.read_csv(self.config.input_data_path)
            contexts = []

            print(f"Loading {min(self.config.sample_size, len(data))} contexts from {self.config.input_data_path}")

            # Limit to sample size
            for _, row in data.head(self.config.sample_size).iterrows():
                # Parse activities from JSON string if stored that way
                activities = row['recent_activities']
                if isinstance(activities, str):
                    try:
                        # Assuming activities are stored as a JSON list string
                        activities = json.loads(activities)
                        # Ensure it's a list, handle cases where it might be a simple string
                        if not isinstance(activities, list):
                            activities = [str(activities)]  # Treat as a single activity if not a list
                    except json.JSONDecodeError:
                        # Handle cases where it's a simple string that isn't JSON
                        activities = [str(activities)]
                elif not isinstance(activities, list):
                    # Handle case where it's not a string or list (e.g., NaN)
                    activities = []

                context = PersonalizedContext(
                    name=row['name'],
                    email=row['email'],
                    job_position=row['job_position'],
                    recent_activities=activities
                )
                contexts.append(context)

            if not contexts:
                print("No contexts loaded. Generating sample contexts.")
                return self._generate_sample_contexts()

            return contexts

        except Exception as e:
            print(f"Error loading contexts from CSV: {e}")
            print("Generating sample contexts for demonstration...")
            return self._generate_sample_contexts()

    def _generate_mock_data(self, num_samples: int):
      """Generate mock data file with Faker."""
      fake = Faker()

      print("Generating synthetic personalized contexts...")

      # Number of records to generate (fixed to 100)

      # Set seeds for reproducibility
      random.seed(42)
      np.random.seed(42)
      Faker.seed(42)

      # Define common job positions
      job_positions = [
        "Software Engineer", "Product Manager", "Marketing Specialist",
        "HR Manager", "Financial Analyst", "Sales Representative",
        "Customer Support", "Data Scientist", "IT Administrator",
        "Project Manager", "Operations Manager", "Executive Assistant",
        "UX Designer", "DevOps Engineer", "Cybersecurity Analyst",
        "Business Analyst", "Legal Consultant", "Recruiter",
        "Quality Assurance Engineer", "Technical Writer", "AI Researcher",
        "Cloud Solutions Architect", "Network Engineer", "Growth Manager",
        "Mobile App Developer", "Systems Analyst", "Machine Learning Engineer",
        "Corporate Trainer", "Content Strategist", "Public Relations Officer",
        "Procurement Specialist", "Risk Manager", "Compliance Officer",
        "Information Security Officer", "Facilities Manager", "Product Designer",
        "Front-End Developer", "Back-End Developer", "Full Stack Developer",
        "Customer Success Manager"
      ]

      # Define common activity templates
      activity_templates = [
        "Working on the {} project",
        "Preparing for the {} presentation",
        "Reviewing {} documents",
        "Attending {} meeting",
        "Planning the next {} initiative",
        "Analyzing {} data trends",
        "Coordinating with the {} team",
        "Implementing a new {} system",
        "Researching {} solutions",
        "Drafting a {} proposal",
        "Responding to {} inquiries",
        "Conducting {} interviews",
        "Troubleshooting {} issues",
        "Organizing the {} workshop",
        "Setting up {} infrastructure",
        "Reviewing feedback from {} clients",
        "Deploying the latest {} update",
        "Refining the {} workflow",
        "Training new hires on {} tools",
        "Budgeting for the {} campaign",
        "Collaborating with {} partners",
        "Finalizing the {} contract",
        "Writing documentation for {} systems",
        "Prototyping the new {} feature",
        "Debugging {} module integration",
        "Evaluating {} vendor performance",
        "Optimizing {} pipeline efficiency"
      ]

      # Company domains
      domains = ["company.com", "enterprise.org", "techcorp.io", "globalfirm.co", "industryco.net"]

      # Generate data
      data = []
      for _ in range(num_samples):
          first_name = fake.first_name()
          last_name = fake.last_name()
          full_name = f"{first_name} {last_name}"

          domain = random.choice(domains)
          # Create plausible email
          email = f"{first_name.lower()}.{last_name.lower()}@{domain}"
          if random.random() < 0.2:  # Occasionally use a different format
              email = f"{first_name.lower()}{last_name.lower()[0]}@{domain}"

          job_position = random.choice(job_positions)

          # Generate 1-3 activities
          num_activities = random.randint(1, 3)
          activities = []
          for _ in range(num_activities):
              activity_template = random.choice(activity_templates)
              activity = activity_template.format(fake.bs())  # Use fake business phrases
              activities.append(activity)

          entry = {
              "name": full_name,
              "email": email,
              "job_position": job_position,
              "recent_activities": json.dumps(activities)
          }

          data.append(entry)

      # Create DataFrame and save to CSV
      df = pd.DataFrame(data)
      df.to_csv(self.config.input_data_path, index=False)

      print(f"Generated {num_samples} mock contexts and saved to {self.config.input_data_path}")

    def run_experiment(self) -> pd.DataFrame:
        """Run the full experiment pipeline: load, generate, detect, save."""
        # Step 1: Load or generate contexts
        print("\n--- Step 1: Loading personalized contexts ---")
        contexts = self.load_contexts()
        if not contexts:
            print("No contexts available to process. Exiting.")
            return pd.DataFrame()  # Return empty DataFrame

        print(f"Loaded {len(contexts)} contexts for processing")

        # Show a few examples
        print("\nExample contexts:")
        for i, context in enumerate(contexts[:min(len(contexts), 3)]):  # Show up to 3 examples
            print(f"\nContext {i+1}:")
            print(f"  Name: {context.name}")
            print(f"  Job: {context.job_position}")
            print(f"  Activities: {', '.join(context.recent_activities) if context.recent_activities else 'None'}")

        results = []

        # Step 2: Generate and detect emails
        print("\n--- Step 2: Generating and detecting phishing emails ---")
        processed_count = 0
        for i, context in enumerate(contexts):
            print(f"\nProcessing context {i+1}/{len(contexts)}: {context.name}")

            # Generate phishing email
            print(f"  Generating phishing email via fine-tuned LLM...")
            phishing_email = self.generator.generate_phishing_email(context)

            # --- Increment counter ---
            processed_count += 1
            print(processed_count)

            if not phishing_email or "Error:" in phishing_email:
                print(f"  Generation failed for {context.name}. Skipping detection.")
                result = {
                    "name": context.name,
                    "email": context.email,
                    "job_position": context.job_position,
                    "recent_activities": context.recent_activities,
                    "generated_email": phishing_email,  # Store error message
                    "model1_pred": False,  # Default values for failed generation
                    "model1_score": 0.0,
                    "model2_pred": False,
                    "model2_score": 0.0,
                    "model3_pred": False,
                    "model3_score": 0.0,
                    "true_label": True  # Still a phishing attempt conceptually
                }
                results.append(result)
                continue

            # Display truncated email preview
            preview = phishing_email.replace('\n', ' ').strip()
            preview = (preview[:150] + '...') if len(preview) > 150 else preview
            print(f"  Email preview: \"{preview}\"")

            # Detect if it's phishing with all models
            print(f"  Running phishing detection with multiple models...")
            detection_results = self.detector.detect_phishing(phishing_email)

            # Print a summary of detection results
            models_detected = sum(1 for k, v in detection_results.items() if k.endswith('_pred') and v)
            print(f"  Detection summary: {models_detected}/3 models identified as phishing")

            # Create result dictionary with all detection results
            result = {
                "name": context.name,
                "email": context.email,
                "job_position": context.job_position,
                "recent_activities": context.recent_activities,
                "generated_email": phishing_email,
                "true_label": True,  # We know it's phishing since we generated it
                **detection_results  # Unpack all detection results
            }
            results.append(result)

        if not results:
            print("No emails were generated successfully to analyze.")
            return pd.DataFrame()

        # Step 3: Create DataFrame and save results
        print("\n--- Step 3: Saving and analyzing results ---")
        results_df = pd.DataFrame(results)
        results_df.to_csv(self.config.results_path, index=False)
        print(f"Results saved to {self.config.results_path}")

        # Print detection summary for all models
        valid_emails = results_df[~results_df['generated_email'].str.contains("Error:", na=False)]
        if not valid_emails.empty:
            print("\n--- Detection Performance Summary ---")
            print(f"Total valid emails: {len(valid_emails)}")
            for model_num in range(1, 4):
                model_col = f"model{model_num}_pred"
                if model_col in valid_emails.columns:
                    detection_rate = valid_emails[model_col].mean() * 100
                    print(f"Model {model_num} detection rate: {detection_rate:.2f}%")

        return results_df

def main():
    """Main function to run the pipeline and evaluate results."""
    print("Starting Fine-Tuned LLM Phishing Email Generation and Detection Pipeline...")

    # Initialize configuration
    config = PipelineConfig()

    # Set a smaller sample size for testing
    config.sample_size = 100  # Generate fewer emails for testing

    # Create experiment runner
    runner = ExperimentRunner(config)

    # Run the experiment
    print("\nRunning experiment...")
    results_df = runner.run_experiment()

    if results_df.empty or results_df[~results_df['generated_email'].str.contains("Error:", na=False)].empty:
        print("\nExperiment finished but no valid emails were generated or processed for analysis.")
        return

if __name__ == "__main__":
    main()

Starting Fine-Tuned LLM Phishing Email Generation and Detection Pipeline...
Loading fine-tuned LLM model: unsloth/Qwen3-0.6B-unsloth-bnb-4bit with adapter: kxm1k4m1/Qwen3-0.6B-unsloth-bnb-4bit-Phishing


tokenizer_config.json:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/4.67k [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/865 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/40.4M [00:00<?, ?B/s]

Successfully loaded fine-tuned model and adapter

Loading detection model1: dima806/phishing-email-detection...
Successfully loaded dima806/phishing-email-detection to cuda

Loading detection model2: cybersectony/phishing-email-detection-distilbert_v2.4.1...
Successfully loaded cybersectony/phishing-email-detection-distilbert_v2.4.1 to cuda

Loading detection model3: ealvaradob/bert-finetuned-phishing...
Successfully loaded ealvaradob/bert-finetuned-phishing to cuda

Running experiment...

--- Step 1: Loading personalized contexts ---
Loading 10 contexts from personalized_contexts.csv
Loaded 10 contexts for processing

Example contexts:

Context 1:
  Name: Danielle Johnson
  Job: Recruiter
  Activities: Implementing a new empower interactive e-services system

Context 2:
  Name: Donald Garcia
  Job: Facilities Manager
  Activities: Training new hires on extend e-business applications tools

Context 3:
  Name: Robert Johnson
  Job: Sales Representative
  Activities: Implementing a new a